# Get All EC2 Encrypted EBS Volume

In [ ]:
import pandas as pd
import boto3
from botocore.config import Config
from IPython.display import display

# 0. Core Function

### Credential Part

In [ ]:
def createAWSCLISession(aws_access_key_id="", aws_secret_access_key="", aws_session_token="", credential_profile=""):
    if aws_access_key_id != "" and aws_secret_access_key != "":
        # Can Specific Key Here
        session = boto3.Session(
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            aws_session_token=aws_session_token
        )

    if credential_profile != "":
        session = boto3.Session(profile_name=credential_profile)
    
    return session

### EC2 Instances

In [ ]:
def extactEC2Instance(reservation=[]):
    return [{"instanceId":i["InstanceId"], "volume": i["BlockDeviceMappings"]} for i in reservation["Instances"]]

In [ ]:
def getAllEBSVolumeEC2InstanceIDByReservations(InstanceIds=[]):
    Reservations = EC2.describe_instances(InstanceIds=InstanceIds).get("Reservations")
    return { res["ReservationId"]:extactEC2Instance(reservation=res) for res in Reservations}

# 1. Check EC2 Encrypted EBS Volume of One Account

In [ ]:
credential_profiles = [
    "<Profile Name>"
]

In [ ]:
credential_profile = credential_profiles[0]
region_name = 'ap-southeast-1'
EC2config = Config(
    region_name = region_name
)
credential_profile

In [ ]:
session = createAWSCLISession(credential_profile=credential_profile)
EC2 = session.client('ec2', config=EC2config)

In [ ]:
InstanceIds = [
    'i-xxxxxxxxxxxxxxx',
    'i-xxxxxxxxxxxxxxx'
]

In [ ]:
ec2_ebs_reserved = getAllEBSVolumeEC2InstanceIDByReservations(InstanceIds=InstanceIds)

In [ ]:
VolumeIds = []
for k, instances in ec2_ebs_reserved.items():
    print("\nReservation ID : "  + str(k))
    for index, instance in enumerate(instances): 
        print(str(index+1) + ". Instance Id : " + instance["instanceId"])
        
        result_df = pd.concat([pd.DataFrame(instance["volume"]), pd.DataFrame(instance["volume"])["Ebs"].apply(pd.Series)], axis=1)
        result_df = result_df.drop(columns="Ebs")
        display(result_df)   
        print("Attached Volume : ", end='')
        VolumeIds += list(result_df["VolumeId"])

In [ ]:
Volumes = EC2.describe_volumes(VolumeIds=VolumeIds).get("Volumes")
pd.DataFrame(Volumes).drop(columns="Attachments")